In [10]:
### Run this cell before continuing.

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

# Function needed to visualize images
# code below sourced from: https://gist.github.com/daviddalpiaz/ae62ae5ccd0bada4b9acd6dbc9008706
def show_digit(arr784):
    plt.imshow(np.array(arr784)[1:].reshape(28, 28), cmap="gray")

In [11]:
titles = ["mean_integrated_profile", "standard_dev_integrated_profile", "excess_kurtosis_integrated_profile", "skewness_integrated_profile",
         "mean_DM-SNR_curve", "standard_dev_DM-SNR_curve", "excess_kurtosis_DM-SNR_curve", "skewness_DM-SNR_curve", "class"]

TEMP_DATA_NAME = pd.read_csv("https://raw.githubusercontent.com/fyip3/ds_project/main/data/HTRU_2.csv", names=titles, on_bad_lines="skip")

In [12]:
TEMP_DATA_NAME

,mean_integrated_profile,standard_dev_integrated_profile,excess_kurtosis_integrated_profile,skewness_integrated_profile,mean_DM-SNR_curve,standard_dev_DM-SNR_curve,excess_kurtosis_DM-SNR_curve,skewness_DM-SNR_curve,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
...,...,...,...,...,...,...,...,...,...
17893,136.429688,59.847421,-0.187846,-0.738123,1.296823,12.166062,15.450260,285.931022,0
17894,122.554688,49.485605,0.127978,0.323061,16.409699,44.626893,2.945244,8.297092,0
17895,119.335938,59.935939,0.159363,-0.743025,21.430602,58.872000,2.499517,4.595173,0
17896,114.507812,53.902400,0.201161,-0.024789,1.946488,13.381731,10.007967,134.238910,0
